In [1]:
import json
import tensorflow as tf
from keras.optimizers import *

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

2.0.0-beta0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

--2019-11-04 07:51:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:4005:802::2010, 172.217.0.48
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:4005:802::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  2.14MB/s    in 2.5s    

2019-11-04 07:51:34 (2.14 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [2]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000
batch_size=32
epochs=50

with open("./sarcasm_data.json", 'r') as f:
    datastore = json.load(f)


sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print("training_padded: "+str(training_padded.shape))
print(str(training_padded))

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print("testing_padded: "+str(testing_padded.shape))
print(str(testing_padded))


training_padded: (20000, 120)
[[328   1 799 ...   0   0   0]
 [  4   1   1 ...   0   0   0]
 [153 890   2 ...   0   0   0]
 ...
 [ 79   1   1 ...   0   0   0]
 [ 53   1   1 ...   0   0   0]
 [312 705   1 ...   0   0   0]]
testing_padded: (6709, 120)
[[  1   1   1 ...   0   0   0]
 [202   1   8 ...   0   0   0]
 [ 18 380 191 ...   0   0   0]
 ...
 [  1   9  67 ...   0   0   0]
 [  1 374   1 ...   0   0   0]
 [  1   1   6 ...   0   0   0]]


In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [4]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 64)           12544     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120, 64)           24832     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 86,657
Trainable params: 86,657
Non-trainable params: 0
____________________________________________________

In [5]:
history = model.fit(training_padded, 
                    training_labels, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(testing_padded, testing_labels),
                    verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 20000 samples, validate on 6709 samples
Epoch 1/50
20000/20000 [==============================] - 140s 7ms/sample - loss: 0.4476 - accuracy: 0.7798 - val_loss: 0.3884 - val_accuracy: 0.8223
Epoch 2/50
20000/20000 [==============================] - 136s 7ms/sample - loss: 0.3546 - accuracy: 0.8381 - val_loss: 0.3737 - val_accuracy: 0.8290
Epoch 3/50
20000/20000 [==============================] - 131s 7ms/sample - loss: 0.3262 - accuracy: 0.8553 - val_loss: 0.3937 - val_accuracy: 0.8247
Epoch 4/50
20000/20000 [==============================] - 131s 7ms/sample - loss: 0.3089 - accuracy: 0.8649 - val_loss: 0.3911 - val_accuracy: 0.8295
Epoch 5/50
20000/20000 [==============================] - 129s 6ms/sample - loss: 0.3651 - accuracy: 0.8381 - val_loss: 0.4033 - val_accuracy: 0.8162
Epoch 6/50
20000/20000 [==============================] - 132s 7ms/sample - loss: 0.3205 - accuracy: 0.8576

In [6]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'acc')
plot_graphs(history, 'loss')

KeyError: 'acc'

In [ ]:
model.save("test.h5")